In [41]:
# fmt: off
import urllib
from IPython.display import Markdown as md

### change to reflect your notebook
_nb_loc = "09_deploying/09c_changesig.ipynb"
_nb_title = "Changing signatures of exported model"

_icons=["https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png", "https://www.tensorflow.org/images/colab_logo_32px.png", "https://www.tensorflow.org/images/GitHub-Mark-32px.png", "https://www.tensorflow.org/images/download_logo_32px.png"]
_links=["https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?" + urllib.parse.urlencode({"name": _nb_title, "download_url": "https://github.com/takumiohym/practical-ml-vision-book-ja/raw/master/"+_nb_loc}), "https://colab.research.google.com/github/takumiohym/practical-ml-vision-book-ja/blob/master/{0}".format(_nb_loc), "https://github.com/takumiohym/practical-ml-vision-book-ja/blob/master/{0}".format(_nb_loc), "https://raw.githubusercontent.com/takumiohym/practical-ml-vision-book-ja/master/{0}".format(_nb_loc)]
md("""<table class="tfo-notebook-buttons" align="left"><td><a target="_blank" href="{0}"><img src="{4}"/>Run in Vertex AI Workbench</a></td><td><a target="_blank" href="{1}"><img src="{5}" />Run in Google Colab</a></td><td><a target="_blank" href="{2}"><img src="{6}" />View source on GitHub</a></td><td><a href="{3}"><img src="{7}" />Download notebook</a></td></table><br/><br/>""".format(_links[0], _links[1], _links[2], _links[3], _icons[0], _icons[1], _icons[2], _icons[3]))
# fmt: on

<table class="tfo-notebook-buttons" align="left"><td><a target="_blank" href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?name=Changing+signatures+of+exported+model&download_url=https%3A%2F%2Fgithub.com%2Ftakumiohym%2Fpractical-ml-vision-book-ja%2Fraw%2Fmaster%2F09_deploying%2F09c_changesig.ipynb"><img src="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png"/>Run in Vertex AI Workbench</a></td><td><a target="_blank" href="https://colab.research.google.com/github/takumiohym/practical-ml-vision-book-ja/blob/master/09_deploying/09c_changesig.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a></td><td><a target="_blank" href="https://github.com/takumiohym/practical-ml-vision-book-ja/blob/master/09_deploying/09c_changesig.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td><td><a href="https://raw.githubusercontent.com/takumiohym/practical-ml-vision-book-ja/master/09_deploying/09c_changesig.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a></td></table><br/><br/>

# エクスポートされたモデルのSignatureの変更  

このノートブックでは、学習済みでエクスポートされたモデルから始めます。  
このモデルはパブリックの`gs://practical-ml-vision-book-data/flowers_5_trained`に保存されています。

## 環境設定
以下に使用するプロジェクト名を入力してください。

In [ ]:
PROJECT = 'PROJECT_NAME'

Colab環境を利用する場合、以下で必要なファイルのダウンロード、プロジェクトの設定を行ってください。

In [ ]:
import os

IS_COLAB_BACKEND = 'COLAB_RELEASE_TAG' in os.environ  # this is always set on Colab
if IS_COLAB_BACKEND:
    from google.colab import auth
    auth.authenticate_user()

    # Get files
    BASE_PATH = "https://raw.githubusercontent.com/takumiohym/practical-ml-vision-book-ja/main/09_deploying"
    !wget {BASE_PATH}/vertex_deploy.sh
    !sudo chmod 755 ./vertex_deploy.sh

    !gcloud config set project {PROJECT}


## Google Cloud Storageバケットの作成
以下のBUCKETの値を使用可能なGSCバケット名に変更して進めてください。<br>
使用できるバケットがない場合は、以下の二行目のコマンドをコメントアウトし、gsutil mbコマンドを実行して作成してください。

In [ ]:
BUCKET='BUCKET_NAME' # Specify your GCS bucket name
# !gsutil mb -l us-central1 -p {PROJECT} gs://{BUCKET} 

## モデルの読み込み

エクスポートされたモデルを読み込みます。


In [1]:
MODEL_LOCATION='gs://practical-ml-vision-book-data/flowers_5_trained'

In [2]:
!gsutil ls {MODEL_LOCATION}

gs://practical-ml-vision-book-data/flowers_5_trained/saved_model.pb
gs://practical-ml-vision-book-data/flowers_5_trained/chkpts/
gs://practical-ml-vision-book-data/flowers_5_trained/variables/


In [3]:
!saved_model_cli show --tag_set serve --signature_def serving_default --dir {MODEL_LOCATION}

2022-07-17 21:27:44.322073: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-17 21:27:44.322125: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
The given SavedModel SignatureDef contains the following input(s):
  inputs['filenames'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_filenames:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['flower_type_int'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall:0
  outputs['flower_type_str'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:1
  outputs['probability'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: StatefulPartitionedCall:2
Meth

## パススルーパラメータを追加する  

上記のSignatureは、入力に利用したファイル名を出力しません。パススルーパラメータとして追加してみましょう。

In [4]:
import os
import shutil
import tensorflow as tf
model = tf.keras.models.load_model(MODEL_LOCATION)
old_fn = model.signatures['serving_default']


@tf.function(input_signature=[tf.TensorSpec([None,], dtype=tf.string)])
def predict_flower_type(filenames):
    result = old_fn(filenames) # has flower_type_int etc.
    result['filename'] = filenames
    return result

shutil.rmtree('export', ignore_errors=True)
os.mkdir('export')
model.save('export/flowers_model',
          signatures={
              'serving_default': predict_flower_type
          })

2022-07-17 21:27:47.488458: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-17 21:27:47.488510: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2022-07-17 21:27:49.654406: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-17 21:27:49.654453: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-17 21:27:49.654480: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tpu-direct): /proc/driver/nvidia/version does not exist
2022-07-17 21:27:49.654756: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-17 21:27:57.425630: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, b

INFO:tensorflow:Assets written to: export/flowers_model/assets


In [5]:
!saved_model_cli show --tag_set serve --signature_def serving_default --dir export/flowers_model

2022-07-17 21:28:01.865116: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-17 21:28:01.865166: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
The given SavedModel SignatureDef contains the following input(s):
  inputs['filenames'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_filenames:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['filename'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:0
  outputs['flower_type_int'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall:1
  outputs['flower_type_str'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:2
  outp

In [6]:
serving_fn = tf.keras.models.load_model('export/flowers_model').signatures['serving_default']
filenames = [
    'gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg',
    'gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg',
    'gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg',
    'gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg',
    'gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg'
]
pred = serving_fn(tf.convert_to_tensor(filenames))
print(pred)

{'probability': <tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.61915255, 0.9999844 , 0.99508286, 0.9751858 , 0.954917  ],
      dtype=float32)>, 'flower_type_str': <tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'dandelion', b'dandelion', b'daisy', b'tulips', b'tulips'],
      dtype=object)>, 'filename': <tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg',
       b'gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg',
       b'gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg',
       b'gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg',
       b'gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg'],
      dtype=object)>, 'flower_type_int': <tf.Tensor: shape=(5,), dtype=int64, numpy=a

出力に、入力したファイル名が合わせて出力されていることが確認できます。

## 複数のSignature

続けて、複数のSignatureを定義する方法を確認します。

In [7]:
@tf.function(input_signature=[tf.TensorSpec([None,], dtype=tf.string)])
def pass_through_input(filenames):
    result = old_fn(filenames) # has flower_type_int etc.
    result['filename'] = filenames
    return result

shutil.rmtree('export', ignore_errors=True)
os.mkdir('export')
model.save('export/flowers_model2',
          signatures={
              'serving_default': old_fn,
              'input_pass_through': pass_through_input
          })

INFO:tensorflow:Assets written to: export/flowers_model2/assets


In [8]:
!saved_model_cli show --tag_set serve --dir export/flowers_model2

2022-07-17 21:28:17.227397: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-17 21:28:17.227450: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "input_pass_through"
SignatureDef key: "serving_default"


`serving_default`シグネチャは先程と同様です。

In [9]:
!saved_model_cli show --tag_set serve --dir export/flowers_model2 --signature_def serving_default

2022-07-17 21:28:20.193490: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-17 21:28:20.193548: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
The given SavedModel SignatureDef contains the following input(s):
  inputs['filenames'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_filenames:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['flower_type_int'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall_1:0
  outputs['flower_type_str'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall_1:1
  outputs['probability'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: StatefulPartitionedCall_1:

２つ目の`input_pass_through`シグネチャを確認します

In [10]:
!saved_model_cli show --tag_set serve --dir export/flowers_model2 --signature_def input_pass_through

2022-07-17 21:28:23.166491: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-17 21:28:23.166543: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
The given SavedModel SignatureDef contains the following input(s):
  inputs['filenames'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: input_pass_through_filenames:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['filename'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:0
  outputs['flower_type_int'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall:1
  outputs['flower_type_str'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:2
  o

## 複数のSignatureを持つモデルをREST APIとしてデプロイ

In [12]:
!gsutil -m cp -r ./export/flowers_model2 gs://{BUCKET}/flowers_model2

Copying file://./export/flowers_model2/keras_metadata.pb [Content-Type=application/octet-stream]...
Copying file://./export/flowers_model2/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://./export/flowers_model2/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://./export/flowers_model2/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
/ [4/4 files][ 11.0 MiB/ 11.0 MiB] 100% Done                                    
Operation completed over 4 objects/11.0 MiB.                                     


モデルをデプロイします。この処理には10分ほど時間がかかります。

In [13]:
%%bash
./vertex_deploy.sh \
--endpoint_name=multi \
--model_name=multi \
--model_location=gs://${BUCKET}/flowers_model2

Deploying model multi
The endpoint named multi already exists.
The endpoint_id is 7785553875351633920
The model named multi already exists.
The model_id is 6103305482900865024
Deploying model now


Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [1042486075676688384]...
.............................................................................................................................................................................................................................................................................................................................................................................................done.
Deployed a model to the endpoint 7785553875351633920. Id of the deployed model: 4075691692072632320.


## 以下のセルを変更する
上記のコマンドから出力された`endpoint_id`を下記にコピーしてください。<br>

In [14]:
# CHANGE THESE TO REFLECT WHERE YOU DEPLOYED THE MODEL
os.environ['ENDPOINT_ID'] = '7785553875351633920' # CHANGE
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = 'us-central1'

### `serving_default`の呼び出し
`serving_default`を呼び出す場合は、[`09b_rest.ipynb`](https://github.com/takumiohym/practical-ml-vision-book-ja/blob/main/09_deploying/09b_rest.ipynb)と同様に、JSONを定義し、`gcloud ai endpoints predict`コマンドを使用します。

In [15]:
%%writefile request.json
{
    "instances": [
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
        }
    ]
}

Overwriting request.json


In [16]:
%%bash
gcloud ai endpoints predict ${ENDPOINT_ID} \
--region=${REGION} \
--json-request=request.json \
--format=json

{
  "deployedModelId": "4075691692072632320",
  "model": "projects/849204435784/locations/us-central1/models/6103305482900865024",
  "modelDisplayName": "multi",
  "predictions": [
    {
      "flower_type_int": 1,
      "flower_type_str": "dandelion",
      "probability": 0.619152546
    },
    {
      "flower_type_int": 1,
      "flower_type_str": "dandelion",
      "probability": 0.999984384
    },
    {
      "flower_type_int": 0,
      "flower_type_str": "daisy",
      "probability": 0.995082855
    },
    {
      "flower_type_int": 4,
      "flower_type_str": "tulips",
      "probability": 0.975185812
    },
    {
      "flower_type_int": 4,
      "flower_type_str": "tulips",
      "probability": 0.954917
    }
  ]
}


Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]


### ２つ目以降のシグネチャの利用
`serving_default`以外のシグネチャを利用する場合以下のことを行う必要があります。

- JSONファイルに`signature_name`キーを追加
- `gcloud ai endpoints raw-predict`コマンドで呼び出し

現在Vertex AIでは、`gcloud ai endpoints predict`ではなく、`gcloud ai endpoints raw-predict`コマンドで`signature_name`キーをサポートしています。

In [17]:
%%writefile request.json
{
    "signature_name": "input_pass_through",
    "instances": [
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
        }
    ]
}

Overwriting request.json


In [18]:
%%bash
gcloud ai endpoints raw-predict ${ENDPOINT_ID} \
--region=${REGION} \
--request=@request.json

{
    "predictions": [
        {
            "filename": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg",
            "flower_type_str": "dandelion",
            "probability": 0.619152546,
            "flower_type_int": 1
        },
        {
            "filename": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg",
            "flower_type_str": "dandelion",
            "probability": 0.999984384,
            "flower_type_int": 1
        },
        {
            "filename": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg",
            "flower_type_str": "daisy",
            "probability": 0.995082855,
            "flower_type_int": 0
        },
        {
            "filename": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg",
            "flower_type_str": "tulips",
            

Using endpoint [https://us-central1-aiplatform.googleapis.com/]


また、同様のことは、直接HTTPリクエストを送信する方法でも実行可能です。<br>
[`09b_rest.ipynb`](https://github.com/takumiohym/practical-ml-vision-book-ja/blob/main/09_deploying/09b_rest.ipynb)と異なり、URLの最後が`rawPredict`となっていることに注意してください。

In [19]:
# Invoke from Python.
import json
from oauth2client.client import GoogleCredentials
import requests

PROJECT = os.environ['PROJECT']
REGION = os.environ['REGION']
ENDPOINT_ID = os.environ['ENDPOINT_ID']

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = "https://{}-aiplatform.googleapis.com/v1/projects/{}/locations/{}/endpoints/{}:rawPredict".format(
    REGION, PROJECT, REGION, ENDPOINT_ID)
headers = {"Authorization": "Bearer " + token }
data = {
    "signature_name": "input_pass_through",
    "instances": [
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
        }
    ]
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

b'{\n    "predictions": [\n        {\n            "flower_type_str": "dandelion",\n            "probability": 0.619152546,\n            "flower_type_int": 1,\n            "filename": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg"\n        },\n        {\n            "flower_type_str": "dandelion",\n            "probability": 0.999984384,\n            "flower_type_int": 1,\n            "filename": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"\n        },\n        {\n            "flower_type_str": "daisy",\n            "probability": 0.995082855,\n            "flower_type_int": 0,\n            "filename": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg"\n        },\n        {\n            "flower_type_str": "tulips",\n            "probability": 0.975185812,\n            "flower_type_int": 4,\n            "filename": "gs://practica

## License
Copyright 2022 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.